In [1]:
# pip install --upgrade anthropic langchain langchain-anthropic python-dotenv

In [ ]:
# -*- coding: utf-8 -*-
"""
Chat agent that searches for research papers,
stores each result in a text file, and continues
conversation until the user types 'exit'.
"""

from dotenv import load_dotenv
import os
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, AgentType
from langchain.tools import Tool
from langchain.memory import ConversationBufferMemory

# 1) Load environment variables
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    raise ValueError("Set OPENAI_API_KEY in your environment or .env file")


# 2) Define a tool that searches for papers and saves them
def search_papers_and_store(topic: str, max_results: int = 5) -> str:
    """
    Stub for 'search_papers'. Replace with real API/scraper logic.
    This function also stores each paper summary in 'papers/' folder.
    """
    # Simulated search results
    results = [f"Paper #{i+1} on {topic}" for i in range(max_results)]
    # Ensure storage folder exists
    folder = "papers"
    os.makedirs(folder, exist_ok=True)
    # Write each result to a separate file
    for idx, paper in enumerate(results, start=1):
        safe_topic = topic.replace(" ", "_")
        filename = os.path.join(folder, f"{safe_topic}_{idx}.txt")
        with open(filename, "w", encoding="utf-8") as f:
            f.write(paper)
    # Return the combined results as the tool output
    return "\n".join(results)


# 3) Wrap it as a LangChain Tool
search_papers_tool = Tool(
    name="search_papers",
    func=search_papers_and_store,
    description="Search for recent research papers on a given topic, store each in a folder, and return summaries.",
    return_direct=True,
)

# 4) Instantiate the OpenAI LLM wrapper
llm = ChatOpenAI(
    model_name="gpt-4o-mini",  # or "gpt-4", "gpt-3.5-turbo"
    temperature=0,
    openai_api_key=OPENAI_API_KEY,
)

# 5) (Optional) in-memory chat history
memory = ConversationBufferMemory(memory_key="chat_history")

# 6) Initialize a zero‐shot REACT agent
agent = initialize_agent(
    tools=[search_papers_tool],
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    memory=memory,
    verbose=True,
)

# 7) Interactive loop
if __name__ == "__main__":
    print("Chat agent initialized. Type your queries or 'exit' to quit.")
    while True:
        user_input = input("You: ").strip()
        if user_input.lower() == "exit":
            print("Agent: Goodbye!")
            break
        # Run agent for each query
        response = agent.run(user_input)
        print(f"Agent: {response}\n")

Chat agent initialized. Type your queries or 'exit' to quit.


> Entering new AgentExecutor chain...
I need to find recent research papers related to laptops. This could include topics like laptop design, performance, usage, or any technological advancements. 
Action: search_papers
Action Input: "laptop"
Observation: Paper #1 on laptop
Paper #2 on laptop
Paper #3 on laptop
Paper #4 on laptop
Paper #5 on laptop


> Finished chain.
Agent: Paper #1 on laptop
Paper #2 on laptop
Paper #3 on laptop
Paper #4 on laptop
Paper #5 on laptop



> Entering new AgentExecutor chain...
I need to search for recent research papers to find the first paper and its author. 
Action: search_papers
Action Input: "recent research papers"
Observation: Paper #1 on recent research papers
Paper #2 on recent research papers
Paper #3 on recent research papers
Paper #4 on recent research papers
Paper #5 on recent research papers


> Finished chain.
Agent: Paper #1 on recent research papers
Paper #2 on recent research